In [1]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import *
from mycelia.shared.chain import * 
from mycelia.miner.train import * 
from mycelia.miner.model_io import *
from mycelia.shared.checkpoint import *
import bittensor as bt 

PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
subtensor = bt.Subtensor('test')
path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
miner_config = MinerConfig.from_path(path)

path = "/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml"
vali_config = ValidatorConfig.from_path(path)

rank = 2
wallet, subtensor = setup_chain_worker(miner_config)
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(miner_config)
expert_manager = ExpertManager(miner_config)

2025-12-15 21:56:39 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-15 21:56:39 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)
2025-12-15 21:56:39 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


## Get commit

In [ ]:
commits = get_chain_commits(miner_config, subtensor)

for c, n in commits:
    display(c)

ValidatorChainCommit(model_hash=None, model_version=-1, expert_group=0, miner_seed=0)

ValidatorChainCommit(model_hash='xxx', model_version=94, expert_group=1, miner_seed=3024650)

None

MinerChainCommit(block=6039781, expert_group=0, model_hash='ce806bfb4b65e8d8128fd0a76d55caff19dff2ba7c6043c2')

None

In [20]:
commits[0][0]

ValidatorChainCommit(model_hash=None, model_version=-1, expert_group=0, miner_seed=0)

## Validator load checkpoint 

In [25]:
get_resume_info(rank=0, config=vali_config)

2025-12-15 03:42:40 [info     ] Looking for checkpoint from folder [mycelia.shared.checkpoint] path={PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation')} result=found


(True,
 ModelMeta(global_ver=-1, inner_opt=-1, path=PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalver_-1'), role=None, model_hash=None),
 PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/globalver_-1'))

In [6]:
# --- Get model from chain ---
def scan_chain_for_new_model(
    current_model_meta: ModelMeta | None,
    config: WorkerConfig,
    subtensor: bittensor.Subtensor,
) -> tuple[bool, list[dict]]:
    """
    Returns:
        should_download: True if a newer model (by version) is available and a majority
                         agree on the model_hash among those newer entries.
        download_meta:   list of dicts with fields: uid, ip, port, model_hash, model_version
                         filtered to entries that (a) are newer and (b) match the majority hash.
    """
    commits: tuple[WorkerChainCommit, bittensor.Neuron] = get_chain_commits(config, subtensor)

    max_model_version = max([getattr(c, "model_version", 0) for c, n in commits])
    if current_model_meta is not None:
        logger.info(
            "check model version",
            max_model_version=max_model_version,
            current_model_version=current_model_meta.global_ver,
        )
        max_model_version = max(max_model_version, current_model_meta.global_ver)

    print('A', commits)
    for c in commits:
        print('B', c)
        
    # 0) Download only from validator

    for c, n in commits:
        print('C', n.validator_permit, getattr(c, "model_hash", False))

    # commits = [(c, n) for c, n in commits if n.validator_permit]
    commits = [(c, n) for c, n in commits if getattr(c, "model_hash", False)]
    commits = [(c, n) for c, n in commits if getattr(c, "miner_seed", False)]

    for c in commits:
        print('D', c)


    # 1) collect candidates that are newer than the current version
    most_updated_commits = [(c, n) for c, n in commits if getattr(c, "model_version", 0) >= max_model_version]

    if len(most_updated_commits) == 0:
        return False, []

    # 2) majority filter by model_hash among the newer candidates
    hash_counts = Counter([c.model_hash for c, n in most_updated_commits if getattr(c, "model_hash", False)])
    majority_hash, _count = hash_counts.most_common(1)[0]

    filtered_commits = [(c, n) for c, n in most_updated_commits if getattr(c, "model_hash", False) == majority_hash]
    if not filtered_commits:
        return False, []

    # 3) prepare download_meta for each entry (uid, ip, port, model_hash, model_version)
    download_meta = []
    for commit, neuron in filtered_commits:
        # Only include entries with reachable metadata
        try:
            download_meta.append(
                {
                    "uid": neuron.uid,
                    "ip": neuron.axon_info.ip,
                    "port": neuron.axon_info.port,
                    "model_hash": commit.model_hash,
                    "model_version": commit.model_version,
                    "target_hotkey_ss58": neuron.hotkey,
                }
            )

            logger.info("Append commit", commit=commit)

        except Exception:
            logger.info("Cannot append commit", commit=commit)

    # should_download if there is at least one newer entry agreeing on a majority hash
    should_download = len(download_meta) > 0

    return should_download, download_meta

current_model_meta = ModelMeta()
should_download, download_metas = scan_chain_for_new_model(current_model_meta, miner_config, subtensor)
should_download, download_metas

2025-12-15 21:57:44 [info     ] check model version          [mycelia.shared.checkpoint] current_model_version=0 max_model_version=94
A [(ValidatorChainCommit(model_hash=None, model_version=3, expert_group=0, miner_seed=0, block=None), NeuronInfoLite(hotkey='5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH', coldkey='5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr', uid=16, netuid=348, active=False, stake=τ0.000000000, stake_dict={'5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr': τ0.000000000}, total_stake=τ0.000000000, rank=0.0, emission=0.0, incentive=0.0, consensus=0.0, trust=0.0, validator_trust=0.0, dividends=0.0, last_update=5750573, validator_permit=False, prometheus_info=PrometheusInfo(block=0, version=0, ip='0.0.0.0', port=0, ip_type=0), axon_info=AxonInfo( /ipv4/149.137.225.62:8000, 5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH, 5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr, 10000000 ), pruning_score=0, is_null=False)), (ValidatorChainCommit(model_hash='xxx', model_v

(True,
 [{'uid': 15,
   'ip': '149.137.225.62',
   'port': 8001,
   'model_hash': 'xxx',
   'model_version': 94,
   'target_hotkey_ss58': '5DUqbhkpvth1WzRogVU9NvGeV9bwX3wBHFWHMZk72WEhZjHF'}])

In [8]:
download_meta = download_metas[0]

In [12]:
ip = download_meta.get("ip")
port = download_meta.get("port")
# Best-effort defaults; customize if your API differs
protocol = getattr(getattr(miner_config, "miner", object()), "protocol", "http")
if ip and port:
    url = f"{protocol}://{ip}:{port}/get-checkpoint"
url

'http://149.137.225.62:8001/get-checkpoint'

In [1]:
fetch_model_from_chain(
    current_model_meta=current_model_meta,
    config=miner_config,
    subtensor=subtensor,
    wallet= wallet
)

NameError: name 'fetch_model_from_chain' is not defined